In [ ]:
# %matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import word2vec

from __future__ import unicode_literals

import numpy as np
import string
import unicodedata
import gensim

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.utils.validation import check_is_fitted
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.layers import Dense, Input, Flatten
from keras.layers import Convolution1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense, Merge
from keras.layers import Flatten
from keras.layers.merge import Concatenate
from keras.layers import concatenate

## Read and preprocess data

In [3]:
train_filename = 'data/train.csv'
data_train = pd.read_csv(train_filename, sep='\t')
data_train = data_train.fillna('')
data_train['date'] = pd.to_datetime(data_train['date'])

In [4]:
test_filename = 'data/test.csv'
data_test = pd.read_csv(test_filename, sep='\t')
data_test = data_test.fillna('')
data_test['date'] = pd.to_datetime(data_test['date'])

In [344]:
data_train.columns

Index(['date', 'edited_by', 'job', 'researched_by', 'source', 'state',
       'statement', 'subjects', 'truth'],
      dtype='object')

In [23]:
col_list = ['job', 'edited_by', 'researched_by', 'source', 
                    'state', 'statement', 'subjects']

statement_len_train = []
for i in range(data_train.shape[0]):
    x = []
    statement_len_train_i = []
    for dd in data_train[col_list].iloc[i].values:        
        x.append(' '.join(strip_accents_unicode(clean_str(dd))))
        statement_len_train_i.append(len(' '.join(clean_str(strip_accents_unicode(dd)))))
    phrase = [' '.join(x)]
    statement_len_train.append(sum(statement_len_train_i))
    if i == 0:
        str_list = phrase[:]
        str_list_i = phrase[:]
    else:
        str_list_i = phrase[:]
        str_list = np.append(str_list, str_list_i, axis=0)

texts_train = str_list.copy().tolist()

In [24]:
len(texts_train)

7569

In [25]:
statement_len_test = []
for i in range(data_test.shape[0]):
    x = []
    statement_len_test_i = []
    for dd in data_test[col_list].iloc[i].values:        
        x.append(' '.join(clean_str(strip_accents_unicode(dd))))
        statement_len_test_i.append(len(' '.join(clean_str(strip_accents_unicode(dd)))))
    phrase = [' '.join(x)]
    statement_len_test.append(sum(statement_len_test_i))
    if i == 0:
        str_list = phrase[:]
        str_list_i = phrase[:]
    else:
        str_list_i = phrase[:]
        str_list = np.append(str_list, str_list_i, axis=0)

texts_test = str_list.copy().tolist()

In [50]:
for i, statement in enumerate(data_train['statement']):
    if u'\xe9' in statement:
        print(statement)
        #statement.replace(u'\xe9', ' ')


Ché Guevara "wrote extensively about the superiority of white Europeans over people of African descent."


## Word2vec

## Word2vec embedding

### Clean statements and calculate length of statements

In [21]:
texts_train = []
texts_test = []
statement_len_train = []
statement_len_test = []
stops = set(stopwords.words('english'))   
for statement in data_train['statement']:
    #clean statements
    statement_new = ' '.join(clean_str(statement))
    statement_len_train.append(len(statement_new))
    texts_train.append(statement_new)
for statement in data_test['statement']:
    #clean statements
    statement_new = ' '.join(clean_str(statement))
    statement_len_test.append(len(statement_new))
    texts_test.append(statement_new)    

In [7]:
#MAX_SEQUENCE_LENGTH = int(np.percentile(statement_len_train, q=85))
MAX_SEQUENCE_LENGTH = max(max(statement_len_train), max(statement_len_test))
MAX_NB_WORDS = sum(statement_len_train) + sum(statement_len_test)
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.25

In [ ]:
#MAX_SEQUENCE_LENGTH = int(np.percentile(statement_len_train, q=85))
MAX_SEQUENCE_LENGTH = max(statement_len_train)
MAX_NB_WORDS = sum(statement_len_train) + sum(statement_len_test)
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.25

In [8]:
MAX_NB_WORDS

1283608

In [9]:
MAX_SEQUENCE_LENGTH

345

In [30]:
len(word_index)

9152

In [26]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
#fit index to word
tokenizer.fit_on_texts(texts_train + texts_test)
#convert word list to sequences
sequences_train = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)
#word index in embedding for word list, unique token
word_index = tokenizer.word_index

### Indexing word vectors

https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings

In [11]:
BASE_DIR = '/Users/lingjin/Desktop/étude/data_camp/fake_news/'
EMBEDDING_FILE = BASE_DIR + 'GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)

In [28]:
print('Preparing embedding matrix')

nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM), dtype=np.float32)
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 3881


In [ ]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        
        col_list = ['job', 'edited_by', 'researched_by', 'source', 
                    'state', 'statement', 'subjects']
        
        for i in range(X_df.shape[0]):
            x = []
            for dd in X_df[col_list].iloc[i].values:        
                x.append(' '.join(clean_str(strip_accents_unicode(dd))))
                phrase = [' '.join(x)]
            if i == 0:
                str_list = phrase[:]
                str_list_i = phrase[:]
            else:
                str_list_i = phrase[:]
                str_list = np.append(str_list, str_list_i, axis=0)
        return str_list
        
        X = np.array(str_list)
        
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [36]:
def strip_accents_unicode(s):
    try:
        s = unicode(s, 'utf-8')
    except NameError:  # unicode is a default on python 3
        pass
    if isinstance(s, float) or isinstance(s, int):
        s = u'number'
    else:
        s = unicodedata.normalize('NFD', s) # NFC, or 'Normal Form Composed' returns composed characters, NFD, 'Normal Form Decomposed' gives you decomposed, combined characters.
    s = s.encode('ascii', 'ignore') # encodes a unicode string to ascii and ignores errors
    try:
        s = s.decode("utf-8")
    except TypeError:  
        pass
    return str(s)

In [38]:
strip_accents_unicode(u'\xe9')

'e'

In [39]:
u'\xe9'.encode('ascii', 'ignore')

b''

### Training data, validation data and test data

In [13]:
train_data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [14]:
train_label = data_train['truth']
y_test = data_test['truth']

In [15]:
# split the data into a training set and a validation set
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)
train_data = train_data[indices]
train_label = train_label[indices]
x_train = np.stack([np.stack([embedding_matrix[word] for word in sentence]) for sentence in train_data])
x_test = np.stack([np.stack([embedding_matrix[word] for word in sentence]) for sentence in x_test])
print("x_train static shape:", x_train.shape)
print("x_test static shape:", x_test.shape)
nb_validation_samples = int(VALIDATION_SPLIT * train_data.shape[0])

x_train = x_train[:-nb_validation_samples][:]
y_train = train_label[:-nb_validation_samples][:]
x_val = x_train[-nb_validation_samples:][:]
y_val = train_label[-nb_validation_samples:][:]

x_train static shape: (7569, 345, 300)
x_test static shape: (2891, 345, 300)


### One hot encoding for target

In [16]:
y_train = np_utils.to_categorical(y_train, num_classes=6)
y_test = np_utils.to_categorical(y_test, num_classes=6)
y_val = np_utils.to_categorical(y_val, num_classes=6)

### Parameters

In [17]:
# Model Hyperparameters
filter_sizes = (2, 3, 4)
num_filters = 128
dropout_prob = 0.2
hidden_dims = 50

# Training parameters
batch_size = 64
num_epochs = 5

# Model input
input_shape = (MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)
model_input = Input(shape=input_shape, dtype='float32')

### Embedding layer for Keras

In [221]:
#CNN-static no need to use embedding layer
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### Happy learning-CNN-static

https://github.com/alexander-rakhlin/CNN-for-Sentence-Classification-in-Keras/blob/master/sentiment_cnn.py

In [18]:
subconv = []
for size in filter_sizes:
    conv = Convolution1D(filters=num_filters,
                         kernel_size=size,
                         padding='valid',
                         activation='relu',
                         strides=1)(model_input)
    maxpool = MaxPooling1D(pool_size=2)(conv)
    flat = Flatten()(maxpool)
    subconv.append(flat)
model = concatenate(subconv, axis=1) if len(subconv) > 1 else subconv[0]
model = Dropout(dropout_prob)(model)
model = Dense(hidden_dims, activation='relu', input_dim=300)(model)
model_output = Dense(6, activation='sigmoid', input_dim=300)(model)

model = Model(model_input, model_output)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
print(model.summary())
# fit the model
model.fit(x_train, y_train, 
         validation_data=(x_val, y_val),
         epochs=3)
# evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 345, 300)      0                                            
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 344, 128)      76928       input_1[0][0]                    
____________________________________________________________________________________________________
conv1d_2 (Conv1D)                (None, 343, 128)      115328      input_1[0][0]                    
____________________________________________________________________________________________________
conv1d_3 (Conv1D)                (None, 342, 128)      153728      input_1[0][0]                    
___________________________________________________________________________________________

KeyboardInterrupt: 

In [250]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 20.338983


In [174]:
subconv = []
for size in filter_sizes:
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False))
    model.add(Convolution1D(filters=num_filters,
                         kernel_size=size,
                         padding="valid",
                         activation="relu",
                         strides=1))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    subconv.append(model)
conv = Sequential()
conv.add(Merge(subconv, mode="concat")) if len(subconv) > 1 else subconv[0]
conv.add(Dropout(dropout_prob))
conv.add(Dense(hidden_dims, activation="relu"))
conv.add(Dense(6, activation="sigmoid"))

conv.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc'])
print(conv.summary())
# fit the model
conv.fit([x_train, x_train, x_train], y_train, 
         validation_data=([x_val, x_val, x_val], y_val),
         epochs=3)
# evaluate the model
loss, accuracy = conv.evaluate([x_test, x_test, x_test], y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

ValueError: Layer concatenate_29 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.models.Sequential'>. Full input: [<keras.models.Sequential object at 0x1c376ea6d8>, <keras.models.Sequential object at 0x1c35c47358>, <keras.models.Sequential object at 0x1c36535e80>]. All inputs to the layer should be tensors.

### New FeatureExtractor

In [295]:
def model_cnn():
    # Matrix parameters
    MAX_SEQUENCE_LENGTH = 264
    MAX_NB_WORDS = 671733# Calculate during analyses
    EMBEDDING_DIM = 300
    # Model Hyperparameters
    num_filters = 128
    dropout_prob = 0.2
    hidden_dims = 50
    filter_sizes = (2, 3, 4)
    # Training parameters
    batch_size = 64
    num_epochs = 5
    # Model input
    input_shape = (MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)
    model_input = Input(shape=input_shape, dtype='float32')
    subconv = []
    for size in filter_sizes:
        conv = Convolution1D(filters=num_filters,
                             kernel_size=sz,
                             padding='valid',
                             activation='relu',
                             strides=1)(model_input)
        maxpool = MaxPooling1D(pool_size=2)(conv)
        flat = Flatten()(maxpool)
        subconv.append(flat)
    model = concatenate(subconv, axis=1) if len(subconv) > 1 else subconv[0]
    model = Dropout(dropout_prob)(model)
    model = Dense(hidden_dims, activation='relu', input_dim=300)(model)
    model_output = Dense(6, activation='sigmoid', input_dim=300)(model)
    model = Model(model_input, model_output)
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

In [302]:
y_train.shape

(5677, 6)

In [333]:
# -*- coding: utf-8 -*-
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.utils.validation import check_is_fitted
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Flatten, Merge
from keras.layers import Convolution1D, MaxPooling1D, Embedding, Dropout
from keras.layers import Embedding
from keras.layers import concatenate

def model_cnn():
    # Matrix parameters
    MAX_SEQUENCE_LENGTH = 264
    MAX_NB_WORDS = 671733# Calculate during analyses
    EMBEDDING_DIM = 300
    # Model Hyperparameters
    num_filters = 128
    dropout_prob = 0.2
    hidden_dims = 50
    filter_sizes = (2, 3, 4)
    # Training parameters
    batch_size = 64
    num_epochs = 5
    # Model input
    input_shape = (MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)
    model_input = Input(shape=input_shape, dtype='float32')
    subconv = []
    for size in filter_sizes:
        conv = Convolution1D(filters=num_filters,
                             kernel_size=size,
                             padding='valid',
                             activation='relu',
                             strides=1)(model_input)
        maxpool = MaxPooling1D(pool_size=2)(conv)
        flat = Flatten()(maxpool)
        subconv.append(flat)
    model = concatenate(subconv, axis=1) if len(subconv) > 1 else subconv[0]
    model = Dropout(dropout_prob)(model)
    model = Dense(hidden_dims, activation='relu', input_dim=300)(model)
    model_output = Dense(6, activation='sigmoid', input_dim=300)(model)
    model = Model(model_input, model_output)
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

class Classifier(object):
    
    def __init__(self):
        self.model = model_cnn()
    
    def fit(self, X, y):
        y_hot = np_utils.to_categorical(y, num_classes=6)[:]
        return self.model.fit(X, y_hot, epochs=3)

    def predict(self, X):
        return self.model.predict(X, verbose=0)

    def predict_proba(self, X):
        return self.model.predict_proba(X)
    
    

In [381]:
# -*- coding: utf-8 -*-
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.utils.validation import check_is_fitted
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from sklearn.utils.validation import check_is_fitted
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Flatten, Merge
from keras.layers import Convolution1D, MaxPooling1D, Embedding, Dropout
from keras.layers import Embedding
from keras.layers import concatenate

def clean_str(sentence, stem=False): # choose whether to stemmer
    english_stopwords = set(
        [stopword for stopword in stopwords.words('english')])
    punctuation = set(string.punctuation)
    punctuation.update(["``", "`", "...", "[", "]", ","])
    if stem:
        stemmer = SnowballStemmer('english')
        return list((filter(lambda x: x.lower() not in english_stopwords and
                            x.lower() not in punctuation,
                            [stemmer.stem(t.lower())
                             for t in word_tokenize(sentence)
                             if t.isalpha()])))

    return list((filter(lambda x: x.lower() not in english_stopwords and
                        x.lower() not in punctuation,
                        [t.lower() for t in word_tokenize(sentence)
                         if t.isalpha()])))

def strip_accents_unicode(s):
    try:
        s = unicode(s, 'utf-8')
    except NameError:  # unicode is a default on python 3
        pass
    s = unicodedata.normalize('NFD', s) # NFC, or 'Normal Form Composed' returns composed characters, NFD, 'Normal Form Decomposed' gives you decomposed, combined characters.
    s = s.encode('ascii', 'ignore') # encodes a unicode string to ascii and ignores errors
    s = s.decode("utf-8")
    return str(s)

def import_sentence(X_df):
    '''Preparation of sentence list for embedding matrix
    '''
    texts = []
    #statement_len = []
    stops = set(stopwords.words('english'))   
    for statement in X_df.statement:
        #clean statements
        statement_new = ' '.join(clean_str(strip_accents_unicode(statement)))
        #statement_len.append(len(statement_new))
        texts.append(statement_new)
    return texts

def embedding_matrix(texts, path='.'):
    MAX_SEQUENCE_LENGTH = 264
    MAX_NB_WORDS = 671733# Calculate during analyses
    EMBEDDING_DIM = 300
    #VALIDATION_SPLIT = 0.25 # Choose not to seperate a validation set
    # Create word_index and sequences
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    # Fit index to word
    tokenizer.fit_on_texts(texts)
    # Convert word list to sequences
    sequences = tokenizer.texts_to_sequences(texts)
    # Word index in embedding for word list, unique token
    word_index = tokenizer.word_index
    # Sequence data
    data = pad_sequences(sequences, 
                         maxlen=MAX_SEQUENCE_LENGTH, 
                         padding='post')
    # Import Word2Vec embeddings
    BASE_DIR = '/Users/lingjin/Desktop/étude/data_camp/fake_news/' # '.'
    EMBEDDING_FILE = BASE_DIR + 'GoogleNews-vectors-negative300.bin'
    word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
            binary=True)
    nb_words = min(MAX_NB_WORDS, len(word_index))+1
    # Create embedding matrix
    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM), dtype=np.float32)
    for word, i in word_index.items():
        if word in word2vec.vocab:
            embedding_matrix[i] = word2vec.word_vec(word)
    print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    
    # Import embeddings to data
    X = np.stack([np.stack([embedding_matrix[word] for word in sentence]) for sentence in data])
    return X

class FeatureExtractor(object):
    """Convert a collection of raw documents to a matrix of TF-IDF features. """

    #def __init__(self):

    def fit(self, X_df, y=None):
        '''Import sentences and pass it to embedding matrix
        '''
        return self

    def fit_transform(self, X_df, y=None):
        texts = import_sentence(X_df)
        X = embedding_matrix(texts, path='.')
        return X

    def transform(self, X_df):
        texts = import_sentence(X_df)
        X = embedding_matrix(texts, path='.')
        return X



## References

http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp

http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow

https://github.com/yoonkim/CNN_sentence

http://ben.bolte.cc/blog/2016/gensim.html

https://github.com/chrisjmccormick/inspect_word2vec

https://stackoverflow.com/questions/43776572/visualise-word2vec-generated-from-gensim

http://nbviewer.jupyter.org/github/danielfrg/word2vec/blob/master/examples/word2vec.ipynb

https://rare-technologies.com/word2vec-tutorial/

quora pairs
https://www.kaggle.com/c/quora-question-pairs/discussion/34355

activation function
https://github.com/Kulbear/deep-learning-nano-foundation/wiki/ReLU-and-Softmax-Activation-Functions

fake news
https://github.com/sumeetkr/AwesomeFakeNews

3 filter size
https://github.com/fchollet/keras/issues/6547

Dropout 
https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/